In [1]:
import os
import re
import cv2
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm
from PIL import Image
from collections import Counter

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torch.nn.functional as F

import torchvision.models as models
import torchvision.transforms as transforms

#from torchvision.datasets import MNIST >>> 나중에 mnist pretrained 사용할 때
from torch.nn.utils.rnn import pack_padded_sequence
from torch.utils.data import Dataset, DataLoader#, SubsetRandomSampler

In [2]:
def check_contour(w, h, area, ratio):
    MIN_AREA = 100
    MIN_WIDTH, MIN_HEIGHT = 5, 25
    MIN_RATIO, MAX_RATIO = 0.2, 0.5
    return area > MIN_AREA and w > MIN_WIDTH and h > MIN_HEIGHT and MIN_RATIO < ratio < MAX_RATIO

In [3]:
def is_green(img):
    height, width, channel = img.shape
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # 가우시안 블러 + threshold
    img_blurred = cv2.GaussianBlur(gray, ksize=(3, 3), sigmaX=0)
    img_blur_thresh = cv2.adaptiveThreshold(gray, maxValue=255.0,
                                           adaptiveMethod=cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                           thresholdType=cv2.THRESH_BINARY_INV,
                                           blockSize=19,
                                           C=9)

    # 윤곽찾기
    contours, _ = cv2.findContours(img_blur_thresh,
                                  mode=cv2.RETR_LIST,
                                  method=cv2.CHAIN_APPROX_SIMPLE)
    contours_dict = []
    cnt = 0

    #idx = 0
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        area = w * h
        ratio = w / h
        cx = x + (w / 2),
        cy = y + (h / 2)

        if check_contour(w, h, area, ratio):    # chech the contour is number or character
            cnt += 1
            #name = 'conout' + str(idx)
            contours_dict.append({
            'contour': contour,
            'x1': x,
            'y1': y,
            'x2': x+w,
            'y2': y+h,
            'w': w,
            'h': h,
            'cx': cx,
            'cy': cy
            })

    if len(contours_dict) == 0:
        return True

    if cnt < 3:
        return True
    else:
        return False

    # bbox 중복제거용.
    # test1. 동작 X contours_dict를 우선순위 heap으로 만들고 while문으로 구현
    # test2. 초록색 탐지 -> 초록색 아닌걸로 판명난 걸 이 method로 탐지
    # 02버 1520 제외 초록색은 모두 제외되었음
    """idx = []
    for i in range(len(contours_dict)):
        box1 = contours_dict[i]

        for j in range(i+1, len(contours_dict)):
            box2 = contours_dict[j]

            x_overlap  = max(0, min(box1['x2'], box2['x2']) - max(box1['x1'], box2['x1']))
            y_overlap  = max(0, min(box1['y2'], box2['y2']) - max(box1['y1'], box2['y1']))
            overallaped_area = x_overlap * y_overlap

            if overallaped_area > 3:
                if box1['w']*box1['h'] < box2['w']*box2['h']:
                    del contours_dict[i]
                else:
                    del contours_dict[j]

    for i in range(len(idx)):
        cnt -= 1
        del contours_dict[idx[i][0]]"""

In [4]:
def exclude_data(file_list, path):
    tmp = [name for name in file_list if not any(re.findall(r'바|사|아|자', name))] # taxi 제외
    res = [name for name in tmp if not any(re.findall('[가-힣]+', name[0:2]))]  # 앞에 지역명을 갖고있는 영업용차량 제외

    valid_data = []
    for name in tqdm(res):

        img_pil = Image.open(path+name)
        img_np = np.array(img_pil)
        img = cv2.cvtColor(img_np, cv2.COLOR_RGB2BGR)

        if not is_green(img):
            valid_data.append(name)
    return valid_data

In [5]:
class Vocabulary(object):
    """Simple vocabulary wrapper."""
    def __init__(self):
        self.word2idx = {}
        self.idx2word = {}
        self.idx = 0

    def add_word(self, word):
        if not word in self.word2idx:
            self.word2idx[word] = self.idx
            self.idx2word[self.idx] = word
            self.idx += 1

    def __call__(self, word):
        return self.word2idx[word]

    def __len__(self):
        return len(self.word2idx)

In [6]:
if __name__=='__main__':

    path = 'C:/Users/anjae/Desktop/PerspectiveImages/'
    file_list = os.listdir(path)
    valid_data = exclude_data(file_list, path)    # >>> 812

100%|███████████████████████████████████████████████████████████████████████████████| 981/981 [00:01<00:00, 567.91it/s]


In [7]:
captions = [name[:-4] for name in valid_data] # eliminate the .jpg
counter = Counter()
for cap in captions:
    for token in cap:
        counter.update(token)
        
words = [word for word, cnt in counter.items()]
vocab = Vocabulary()
vocab.add_word('<start>')
vocab.add_word('<end>')

for i, word in enumerate(words):
    vocab.add_word(word)

In [8]:
vocab.idx2word

{0: '<start>',
 1: '<end>',
 2: '0',
 3: '1',
 4: '도',
 5: '9',
 6: '8',
 7: '2',
 8: '두',
 9: '6',
 10: '7',
 11: '3',
 12: '오',
 13: '호',
 14: '4',
 15: '거',
 16: '모',
 17: '버',
 18: '5',
 19: '주',
 20: '고',
 21: '나',
 22: '라',
 23: '로',
 24: '루',
 25: '보',
 26: '부',
 27: '가',
 28: '마',
 29: '머',
 30: '소',
 31: '조',
 32: '하',
 33: '구',
 34: '누',
 35: '러',
 36: '노',
 37: '수',
 38: '우',
 39: '무',
 40: '저',
 41: '더',
 42: '너',
 43: '어',
 44: '다',
 45: '서'}

In [9]:
vocab.word2idx

{'<start>': 0,
 '<end>': 1,
 '0': 2,
 '1': 3,
 '도': 4,
 '9': 5,
 '8': 6,
 '2': 7,
 '두': 8,
 '6': 9,
 '7': 10,
 '3': 11,
 '오': 12,
 '호': 13,
 '4': 14,
 '거': 15,
 '모': 16,
 '버': 17,
 '5': 18,
 '주': 19,
 '고': 20,
 '나': 21,
 '라': 22,
 '로': 23,
 '루': 24,
 '보': 25,
 '부': 26,
 '가': 27,
 '마': 28,
 '머': 29,
 '소': 30,
 '조': 31,
 '하': 32,
 '구': 33,
 '누': 34,
 '러': 35,
 '노': 36,
 '수': 37,
 '우': 38,
 '무': 39,
 '저': 40,
 '더': 41,
 '너': 42,
 '어': 43,
 '다': 44,
 '서': 45}

In [10]:
captions

['01도0982',
 '01두6167',
 '01두6173',
 '01오2026',
 '01호7971',
 '01호9428',
 '02거0688',
 '02모6182',
 '02버1520',
 '02버8171',
 '02주8096',
 '03고2103',
 '03나5394',
 '03라8025',
 '03로2945',
 '03루5237',
 '03버4113',
 '03보8129',
 '03부4568',
 '04가2678',
 '04고8608',
 '04나1851',
 '04마3810',
 '04머0711',
 '04소0381',
 '04소0425',
 '04조5542',
 '04하0493',
 '05가7190',
 '05거0214',
 '05고3250',
 '05구8036',
 '05누6006',
 '05누6109',
 '05누6625',
 '05도1291',
 '05도7067',
 '05라9211',
 '05로6033',
 '05모9326',
 '05소4660',
 '06러9201',
 '06러9329',
 '06루2679',
 '06모1085',
 '06모1297',
 '06모1437',
 '07고4598',
 '07노9076',
 '07두4907',
 '07모3479',
 '07소3599',
 '07수1184',
 '08두5232',
 '08보1242',
 '08보1292',
 '08수7006',
 '08수7040',
 '08우1024',
 '09누6636',
 '09라0676',
 '09라0710',
 '09라0740',
 '09라0798',
 '09러2342',
 '09러2376',
 '09마0604',
 '09머5540',
 '09무3054',
 '09우5531',
 '09저6947',
 '10누5019',
 '10누5053',
 '10더5037',
 '10루4154',
 '10무1590',
 '10부6770',
 '10소2938',
 '10우3213',
 '10조8410',
 '10주0605',
 '11너9153',
 '11노3042',
 '11

In [11]:
# fInd mean and std for each channel for valid_data
imgs = []
for file_name in valid_data:
    img = Image.open(path + file_name)
    imgs.append(np.asarray(img))

imgs = np.asarray(imgs)
normalize_mean = [imgs[:,:,:,0].mean(), imgs[:,:,:,1].mean(), imgs[:,:,:,2].mean()]
normalize_std = [imgs[:,:,:,0].std(), imgs[:,:,:,1].std(), imgs[:,:,:,2].std()]

In [12]:
normalize_mean

[140.35294402260286, 141.85065142150805, 140.9446750838778]

In [13]:
normalize_std

[67.1339938663095, 66.77541666677854, 66.45039705287245]

In [14]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(normalize_mean,normalize_std),
])

In [15]:
class CustomDataset(Dataset):
    def __init__(self, imgs, vocab, captions, transform):
        self.imgs = imgs
        self.captions = captions
        self.vocab = vocab
        self.transform = transform
        
    def __getitem__(self, index):
        imgs = self.imgs
        vocab = self.vocab
        captions = self.captions
        
        image = imgs[index]
        if self.transform is not None:
            image = self.transform(image)
        
        tokens = [token for token in captions[index]]
        caption = []
        caption.append(vocab('<start>'))
        caption.extend([vocab(token) for token in tokens])
        caption.append(vocab('<end>'))
        target = torch.Tensor(caption)
        
        return image, target
    
    def __len__(self):
        return len(self.imgs)

In [16]:
batch_size = 8
shuffle = False

c = CustomDataset(imgs, vocab, captions, transform)
data_loder = DataLoader(dataset=c,
                       batch_size=batch_size,
                       shuffle=shuffle,
                       )

train_size = int(0.8 * c.__len__())
test_size = c.__len__() - train_size

In [17]:
torch.manual_seed(10)
train_dataset, test_dataset = data.random_split(c, [train_size, test_size])
train_loder = data.DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle)
test_loder = data.DataLoader(test_dataset, batch_size=batch_size, shuffle=shuffle)

#### Building Model
- 1. quick train vgg11 with mnist dataset
- 2. connect the feature parts of it to RNN

In [18]:
# Mnist데이터 훈련된 vgg11 모델이 필요하다
# 후에 update
class MnistVgg11():
    def __init__(self):
        self.vgg11 = models.vgg11(pretrained=True)
        self.train_batch_size = 16
        train_loader = DataLoader(MNIST(download=True, root=".", transform=data_transform, train=True),
                              batch_size=train_batch_size, shuffle=True)
        output = self.vgg1(train_loader.sampler.data_source)
        self.vgg11.classifier = nn.Linear(output, 10)
    
    def train(self):
        epoch = 100
        train_loade, val

In [19]:
class EncoderCNN(nn.Module):
    def __init__(self, embed_size):
        super(EncoderCNN, self).__init__()
        self.vgg11 = models.vgg11(pretrained=False)   # 해당 부분 나중에 trained vgg11으로 교체 / feature map을 히든으로 사용 test
        self.vgg11.classifier = self.vgg11.classifier[0:2]
        self.linear = nn.Linear(4096, embed_size)
        self.bn = nn.BatchNorm1d(embed_size)
        
    def forward(self, images):
        """피쳐 벡터 추출 후 RNN의 input 이미지로 사용"""
        features = self.vgg11(images)
        features = self.bn(self.linear(features))
        return features

class DecoderRNN(nn.Module):
    def __init__(self, embed_size, hidden_size, vocab_size, num_layers=1, batch_first=True):
        super(DecoderRNN, self).__init__()
        self.hidden_dim = hidden_size
        self.embed = nn.Embedding(vocab_size, embed_size)  # 46 x 256
        self.batch_first = batch_first
        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # 256 -> 256
        if batch_first:
            self.lstm = nn.LSTM(input_size=embed_size, hidden_size=hidden_size, \
                                num_layers=num_layers, batch_first=batch_first)
        else:
            self.lstm = nn.LSTM(input_size=embed_size, hidden_size=hidden_size, \
                                num_layers=num_layers)
            
        # 256 -> 46
        self.linear = nn.Linear(hidden_size, vocab_size)
        self.dropout = nn.Dropout(0.5)

    def forward(self, feature, caption):
        """CNN으로 얻은 feature 벡터를 복호화하고 캡션을 생성"""
        # caption_size: caption_len -1, batch   or  batch, caption_len
        embeddings = self.embed(caption)
        # caption_size: 256, batch   or  batch, 256
        
        
        # feature: batch x 512 -> [1 x batch x 256]   or   [batch x 1 x 256]
        if self.batch_first:
            embeddings = torch.cat((feature.unsqueeze(1), embeddings), 1)
        else:
            embeddings = torch.cat((feature.unsqueeze(0), embeddings), 0)
        # >>> 9 x batch x 256  or batch x 9 x 256
  
        hiddens, _ = self.lstm(embeddings)
        # 9 x batch x 256   or   batch x 9 x 256
        
        outputs = self.linear(hiddens)
        # 9 x batch x 46     or    batch x 9 x 46
        return outputs
    
class CNN2RNN(nn.Module):
    def __init__(self, embed_size, hidden_size, vocab_size, num_layers=1, batch_first=True):
        super(CNN2RNN, self).__init__()
        self.encoder = EncoderCNN(embed_size)
        self.decoder = DecoderRNN(embed_size, hidden_size, vocab_size, num_layers, batch_first=batch_first)
        self.batch_first = batch_first
        
    def forward(self, x, caption):
        feature = self.encoder(x)   # batch x 256
        output = self.decoder(feature, caption)
        return output
    
    def sample_caption(self, image, vocab):
        result = []
        
        with torch.no_grad():
            feature = self.encoder(image.unsqueeze(0)) # image를 한개씩 받기 때문에 batch차원 늘려준다
            states = None
            
            for _ in range(8):
                if self.batch_first:
                    hiddens, states = self.decoder.lstm(feature.unsqueeze(1), states)
                    output = self.decoder.linear(hiddens.squeeze(0))
                    predicted = output.argmax(1)         
                    result.append(predicted.item())
                    x = self.decoder.embed(predicted).unsqueeze(0)
                else:
                    hiddens, states = self.decoder.lstm(feature.unsqueeze(0), states)
                    output = self.decoder.linear(hiddens.squeeze(0))
                    predicted = ouptut.argmax(1)
                    result.append(predicted.item())
                    x = self.decoder.embed(predicted).unsqueeze(0)
                    
                if vocab.idx2word[predicted.item()] == '<end>':
                    break
                    
        return [vocab.idx2word[i] for i in result]

In [20]:
device='cpu'
lr = 0.001
epoch = 100
embed_size = 256
hidden_size = 256
criterion = nn.CrossEntropyLoss()

#model = CNN2RNN(embed_size, hidden_size, len(vocab), batch_first=False)
model = CNN2RNN(embed_size, hidden_size, len(vocab), batch_first=True)
optimizer = optim.Adam(model.parameters(), lr=lr)

In [22]:
for _ in range(epoch):
    for idx, (image, caption) in tqdm(enumerate(train_loder)):
        image = image.to(device)
        caption = caption.type(torch.LongTensor).to(device)

        output = model(image, caption[:, :-1])   #  batch x step(8) x vacab_size 
        output = output.reshape(-1, output.shape[2])   # (batch*step(8)) x vocab_size

        loss = criterion(output, caption.reshape(-1))

        optimizer.zero_grad()
        loss.backward(loss)
        optimizer.step()
        
        if (idx+1) % 5 == 0:
            print(f'epoch: {_}, step:{idx}, loss: {loss}')
            torch.save(model.state_dict(), f'./save_path/epoch{_}.pth')

4it [00:12,  3.16s/it]

epoch: 0, step:4, loss: 2.382465362548828


9it [00:30,  3.33s/it]

epoch: 0, step:9, loss: 2.3400845527648926


14it [00:48,  3.42s/it]

...

epoch: 66, step:9, loss: 0.2060471922159195


14it [01:20,  5.76s/it]

epoch: 66, step:14, loss: 0.2005997598171234


16it [01:37,  6.09s/it]


KeyboardInterrupt: 

#### Sample Test

In [23]:
sample_img, sample_cap = test_dataset.__getitem__(3)

In [24]:
model.eval()
res = model.sample_caption(sample_img, vocab)

In [25]:
res

['<start>',
 '<start>',
 '<start>',
 '<start>',
 '<start>',
 '<start>',
 '<start>',
 '<start>']

In [26]:
[vocab.idx2word[i.item()] for i in sample_cap]

['<start>', '5', '3', '두', '6', '0', '6', '4', '<end>']

#### .... Need to improve some point or may be it is not valid approach